In [ ]:
!pip install sdv


In [2]:
import sdv
print(sdv.__version__)


1.19.0


In [ ]:
import pandas as pd

df = pd.read_excel('/content/Patient_data.xlsx', header=1)

print(df.columns)
df.head()
df['S.NO'] = df['S.NO'].astype('Int64')  



Index(['S.NO', 'PATIENT NAME', 'AGE', 'SEX', 'DIAGNOSIS', 'LOCATION',
       'level of lymph node', 'ENE', 'Histologic Grading', 'SURGERY',
       'RADIOTHERAPY', 'CHEMOTHERAPY', 'RECURRENCE'],
      dtype='object')


In [7]:
from sdv.metadata import SingleTableMetadata
from sdv.lite import SingleTablePreset

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df)

categorical_columns = [
    'SEX', 'DIAGNOSIS', 'LOCATION', 'level of lymph node',
    'ENE', 'Histologic Grading', 'SURGERY',
    'RADIOTHERAPY', 'CHEMOTHERAPY', 'RECURRENCE'
]
for col in categorical_columns:
    metadata.update_column(column_name=col, sdtype='categorical')

model = SingleTablePreset(name='FAST_ML', metadata=metadata)

model.fit(df)

synthetic_data = model.sample(300)
synthetic_data['S.NO'] = range(1, len(synthetic_data) + 1)
synthetic_data = synthetic_data.fillna({
    'AGE': df['AGE'].median(),
    'SEX': 'M',
    'RECURRENCE': 'N',
    'DIAGNOSIS': df['DIAGNOSIS'].mode()[0],
    'LOCATION': df['LOCATION'].mode()[0],
    'level of lymph node': df['level of lymph node'].mode()[0],
    'ENE': 'N',
    'SURGERY': df['SURGERY'].mode()[0],
    'RADIOTHERAPY': df['RADIOTHERAPY'].mode()[0],
    'CHEMOTHERAPY': df['CHEMOTHERAPY'].mode()[0],
    'Histologic Grading': df['Histologic Grading'].mode()[0],
})
synthetic_data['PATIENT NAME'] = [f"Patient_{i}" for i in range(1, len(synthetic_data) + 1)]


synthetic_data.to_csv("synthetic_patients_300.csv", index=False)

synthetic_data.head()


/usr/local/lib/python3.11/dist-packages/sdv/lite/single_table.py:52: FutureWarning: The 'SingleTablePreset' is deprecated. For equivalent Fast ML functionality, please use the 'GaussianCopulaSynthesizer'.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/lite/single_table.py:61: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(META_DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sdv/lite/single_table.py:100: FutureWarnin

,S.NO,PATIENT NAME,AGE,SEX,DIAGNOSIS,LOCATION,level of lymph node,ENE,Histologic Grading,SURGERY,RADIOTHERAPY,CHEMOTHERAPY,RECURRENCE
0,1,Patient_1,60.433499,M,CA RIGHT LATERAL BORDER OF TONGUE,RIGHT LATERAL BORDER OF TONGUE,LEVEL 2,N,WELL DIFFERENTIATED SCC,Y,Y,N,N
1,2,Patient_2,44.689028,M,CA LEFT MAXILLA,RIGHTBUCCAL MUCOSA,LEVEL 2,N,WELL DIFFERENTIATED SCC,Y,Y,N,N
2,3,Patient_3,44.856399,F,CA RIGHTBUCCAL MUCOSA,LEFT BUCCAL MUCOSA,LEVEL 2,N,WELL DIFFERENTIATED SCC,Y,Y,N,N
3,4,Patient_4,41.037259,M,CA RIGHT MAXILLA,LEFT BUCCAL MUCOSA,LEVEL 3,N,MODERATELY DIFFERENTIATED SCC,Y,Y,N,N
4,5,Patient_5,53.658288,M,CA RIGHT MAXILLA,RIGHT BUCCAL MUCOSA,LEVEL 2,N,MODERATELY DIFFERENTIATED SCC,Y,Y,N,N
